In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch, whisper, re, difflib


whisper_model = whisper.load_model("base")


/home/zayn-husyn/project/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)

In [2]:
import pandas as pd
df = pd.read_csv("dataset.csv", encoding="utf-8")
df.columns = df.columns.str.strip()

insight_dict = {
    str(word).strip().lower(): str(expl).strip()
    for word, expl in zip(df["word"], df["explanation"])
}

cs_keywords = list(insight_dict.keys())

In [3]:
peft_path = "./qwen_lora_finetuned"
peft_config = PeftConfig.from_pretrained(peft_path)

base = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path, trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(peft_path, trust_remote_code=True)
model = PeftModel.from_pretrained(base, peft_path, torch_dtype=torch.float32)
model = model.merge_and_unload().to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
model.eval()

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [4]:

def extract_cs_terms(text):
    return list({term for term in cs_keywords if re.search(rf'\b{re.escape(term)}\b', text.lower())})

def lookup_or_generate(term):
    key = term.lower()
    if key in insight_dict:
        return insight_dict[key]
    m = difflib.get_close_matches(key, insight_dict.keys(), n=1, cutoff=0.85)
    if m:
        return insight_dict[m[0]]
    prompt = f"Explain this computer-science term in simple words:\n\nTerm: {term}\nExplanation:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=80, temperature=0.7,
                         pad_token_id=tokenizer.eos_token_id)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    expl = text.split("Explanation:")[-1].strip().split("\n")[0]
    insight_dict[key] = expl 
    return expl

In [5]:
def audio_to_insights(audio_path):
    res = whisper_model.transcribe(audio_path)
    transcript = res["text"]
    terms = extract_cs_terms(transcript)
    if not terms:
        return transcript, [], ["No CS jargon found."]
    insights = [f" {t}: {lookup_or_generate(t)}" for t in terms]
    return transcript, terms, insights

In [6]:
if __name__ == "__main__":
    path = "Crash Course Computer Science Preview.mp3"
    txt, jargon, outs = audio_to_insights(path)
    print("Transcript:", txt)
    print("Detected jargon:", jargon)
    print("\n".join(outs))

/home/zayn-husyn/project/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcript:  Hello world, I'm Karyan Philbin and welcome to Crash Course Computer Science. So, computers really have allowed us to do some pretty amazing things. Think global telecommunications, international commerce, global transportation, breakthroughs in medicine, distributed education, online shopping, online dating, and just the internet in general. Computers are allowing us to explore our own world and other worlds, and of course some seemingly mundane things like permitting us to smile on our pets from work, or communicating with our friends in a nearly indecisive or voice stream of emoji. But don't call computers magical, they are not. I repeat, are not magical. So before we get into what we're going to talk about in this series, it might be useful to tell you what we are not going to talk about. We aren't going to teach you how to program. Programming is a really crucial aspect of computer science, and we will get to the rules that guide the logic of hardware and software des